# Extracting and Visualizing Stock Data

Importing necessary libraries:

In [9]:
#ifinance for extracting data from yfinance API
import yfinance as yf
# pandas is a default
import pandas as pd
# BeautifulSoup from bs4, for manually scraping tables
from bs4 import BeautifulSoup
# requests for extracting data to use with BeautifulSoup
import requests
# some plotly functions 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio 
# Display, so we can see all the tables in the webpage in a decent manner while looping
from IPython.display import display

## Tesla stocks from yfinance
First we extract tesla stocks from yfinance,
the Ticker symbol is TSLA

In [15]:
tesla = yf.Ticker('TSLA')

In [16]:
tesla_data= tesla.history(period= 'max')

In [17]:
#let's see what we extracted:
tesla_data.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


In [18]:
#Now we adjust index, we want Date as a column
tesla_data.reset_index(inplace=True)
tesla_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


## Now we use webscraping to extract tesla revenues data from the given url:
we are focusing on quarterly revenue

In [29]:
url='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm'

##### Method 1: manual extractiong with beautyful soup

In [30]:
tesla_html= requests.get(url).text
soup=BeautifulSoup(tesla_html, 'html.parser')
tables=soup.find_all('table')
len(tables)

6

We have 6 tables in the web page, opening the page it looks like that:

<details><summary>Click hto see web page image</summary>
<img src="img_htmlpage.png" alt="">

so it looks like we need to extract the second table, but to be sure let's extract the table headers for a better understanding (in the actual web page i can only see 4 tables)

In [31]:
# creatig an empty list of empty lists in order to store the headers
#(we will use later the correct header)
tables_headers=[[] for _ in range(len(tables))]
tables_headers

[[], [], [], [], [], []]

In [32]:
# this is for refreshing my memory about what we extract with find_all
tables[1].find_all('th')

[<th colspan="2" style="text-align:center">Tesla Quarterly Revenue<br/><span style="font-size:14px;">(Millions of US $)</span></th>]

In [33]:
# now we can actually extract the headers: 
for i, table in enumerate(tables): 
    for header in table.find_all('th'):
        tables_headers[i].append(header.text)
# Let's see what we extracted:
for i, _ in enumerate(tables_headers):
    print(f'Table {i} headers: {_}')



Table 0 headers: ['Tesla Annual Revenue(Millions of US $)']
Table 1 headers: ['Tesla Quarterly Revenue(Millions of US $)']
Table 2 headers: ['Sector', 'Industry', 'Market Cap', 'Revenue']
Table 3 headers: ['Stock Name', 'Country', 'Market Cap', 'PE Ratio']
Table 4 headers: ['Link Preview', 'HTML Code (Click to Copy)']
Table 5 headers: ['Link Preview', 'HTML Code (Click to Copy)']


It seems that we were right: we need the second table, so we can reassign variables and dump all the variables that we don't need:

In [34]:
#I am creating the tesla table in this way so we dont't have a direct reference to the original object
tesla = BeautifulSoup(str(tables[1]), 'html.parser')
tesla_headers= tables_headers[1]
del(tables_headers)
tesla_headers

['Tesla Quarterly Revenue(Millions of US $)']

In [35]:
# as we have a single header, we can do as follow: 
tesla_headers=tesla_headers[0]

In [36]:
dict_={0:[], 1: []} #we create a dictionary with 2 keys, because we have seen in the website that the table has 2 columns
for row in tesla.find('tbody').find_all('tr'): 
    for i, col in enumerate(row.find_all('td')): 
        dict_[i].append(col.text)


tesla_revenues=pd.DataFrame(dict_)
#check what we did:
tesla_revenues.head()

,0,1
0,2022-09-30,"$21,454"
1,2022-06-30,"$16,934"
2,2022-03-31,"$18,756"
3,2021-12-31,"$17,719"
4,2021-09-30,"$13,757"


In [38]:
#rename the columns:
tesla_revenues.columns=['Date', tesla_headers ]
tesla_revenues.head()

,Date,Tesla Quarterly Revenue(Millions of US $)
0,2022-09-30,"$21,454"
1,2022-06-30,"$16,934"
2,2022-03-31,"$18,756"
3,2021-12-31,"$17,719"
4,2021-09-30,"$13,757"


In [40]:
# now we have to remove the dollar sign from the column[1]: 
tesla_revenues.iloc[:,1]=tesla_revenues.iloc[:,1].map(lambda x: str(x)[1:])
tesla_revenues.head()


,Date,Tesla Quarterly Revenue(Millions of US $)
0,2022-09-30,"21,454"
1,2022-06-30,"16,934"
2,2022-03-31,"18,756"
3,2021-12-31,"17,719"
4,2021-09-30,"13,757"


## Now we use pandas read_html to extract tesla revenues data from the given url:

In [42]:
tables=pd.read_html(url)

In [43]:
len(tables)

6

In [44]:
for i, table in enumerate(tables): 
   display(table.head())

,Tesla Annual Revenue (Millions of US $),Tesla Annual Revenue (Millions of US $).1
0,2021,"$53,823"
1,2020,"$31,536"
2,2019,"$24,578"
3,2018,"$21,461"
4,2017,"$11,759"


,Tesla Quarterly Revenue (Millions of US $),Tesla Quarterly Revenue (Millions of US $).1
0,2022-09-30,"$21,454"
1,2022-06-30,"$16,934"
2,2022-03-31,"$18,756"
3,2021-12-31,"$17,719"
4,2021-09-30,"$13,757"


,Sector,Industry,Market Cap,Revenue
0,Auto/Tires/Trucks,Auto Manufacturers - Domestic,$549.575B,$53.823B
1,Tesla is the market leader in battery-powered ...,Tesla is the market leader in battery-powered ...,Tesla is the market leader in battery-powered ...,Tesla is the market leader in battery-powered ...


,Stock Name,Country,Market Cap,PE Ratio
0,General Motors (GM),United States,$53.930B,5.56
1,Ford Motor (F),United States,$52.668B,8.09
2,Harley-Davidson (HOG),United States,$6.762B,9.56
3,Polaris (PII),United States,$6.267B,11.86
4,IAA (IAA),United States,$5.134B,16.40


,Link Preview,HTML Code (Click to Copy)
0,Tesla Revenue 2010-2022 | TSLA,NaN
1,Macrotrends,NaN
2,Source,NaN


,Link Preview,HTML Code (Click to Copy)
0,Tesla Revenue 2010-2022 | TSLA,NaN
1,Macrotrends,NaN
2,Source,NaN


As we can see, we need the second table

In [45]:
tesla__revenues_pd=tables[1]
tesla__revenues_pd.head()

,Tesla Quarterly Revenue (Millions of US $),Tesla Quarterly Revenue (Millions of US $).1
0,2022-09-30,"$21,454"
1,2022-06-30,"$16,934"
2,2022-03-31,"$18,756"
3,2021-12-31,"$17,719"
4,2021-09-30,"$13,757"


now we can fix the columns name and the column[1] values

In [48]:
tesla__revenues_pd.iloc[:,1] = tesla__revenues_pd.iloc[:,1].map(lambda x: str(x)[1:])
tesla__revenues_pd.columns= ['Date',tesla__revenues_pd.columns[0]]
tesla__revenues_pd.head()

,Date,Tesla Quarterly Revenue (Millions of US $)
0,2022-09-30,"1,454"
1,2022-06-30,"6,934"
2,2022-03-31,"8,756"
3,2021-12-31,"7,719"
4,2021-09-30,"3,757"
